# P2PKH transaction - BX Exercise
In this exercise, we spend a single P2PKH input to a P2PKH output.

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending + receiving key pairs.


### Restore HD master keys from mnemonic sentence.

In [ ]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)

### Restore your first receiving key in your account 1: `m/44'/1'/1'/0/0`.

(We will be spending the funds controlled by this key)

In [ ]:
hd_m_44h_1h_1h_0_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 0)

privatekey_44h_1h_1h_0_0=$(bx hd-to-ec $hd_m_44h_1h_1h_0_0)
publickey_44h_1h_1h_0_0=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_0=$(bx sha256 $publickey_44h_1h_1h_0_0 | bx ripemd160)
address_44h_1h_1h_0_0=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_0
echo $publickeyhash_44h_1h_1h_0_0


### Fetch previous UTXO you are spending.


In [ ]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"

bx fetch-tx 11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea -f json | jq ".transaction.outputs[0]"

In [ ]:
# previous_txid=
# previous_output_index=
# previous_output_amount=

previous_txid=11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea
previous_output_index=0
previous_output_amount=499543


### Generate destination key pair from third wallet account.

You will spend the previously received utxo to receiving addresses from `account 2`, namely `/44'/1'/2'/0/0`.

In [ ]:
# Derive hd private key m_44h_1h_2h_0_0 from your master keys:

# hd_m_44h_1h_2h_0_0=
# publickeyhash_44h_1h_2h_0_0=

hd_m_44h_1h_2h_0_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 0)

publickeyhash_44h_1h_2h_0_0=$(bx hd-to-public $hd_m_44h_1h_2h_0_0 | bx hd-to-ec | bx sha256 | bx ripemd160)
echo $publickeyhash_44h_1h_2h_0_0

## 2. Building and signing transaction with P2PKH output.

<br>
<img src="images/transaction_signing_bx.jpg" alt="drawing" style="" width="800px"/>

### A. Create P2PKH output script.

In [ ]:
# bx script-encode "DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"

# output_script=

output_script=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_2h_0_0"] EQUALVERIFY CHECKSIG")
echo $output_script

### B. Compute output amount(s).

* **Subtract minimum fee, which consists of:**
    * Example minimum fee levels: `100 sat / sigop + 1 sat / byte`
    * This is node policy. 


* **Transaction sigop count computation:**
    * Counted as 1 sigop: `checksig`,`checksigverify`
    * Counted as 20 sigops: `checkmultisig`, `checkmultisigverify`
    * *Non-segwit:*
        * Transaction sigop count is multiplied by 4x.
        * Output vs Input sigops: 
            * All sigops in the output script are counted.
            * P2SH Redeemscript input sigops are counted.
      

In [ ]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+25) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 148 + 1 + 1 \* 34 + 4)
echo $tx_byte_count


In [ ]:
# Output amount total.

# output_amount=

output_amount=$(expr $previous_output_amount - 1 \* 400 - $tx_byte_count \* 1)
echo $output_amount


### C. Construct transaction template for signing.

* `sequence: 0xffffffff(hex)/4294967295(dec)`

In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# my_tx=

my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script:$output_amount)


### D. Sign transaction.


**Fetch previous output script**

In [ ]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[0].script")
echo $previous_output_script


**Sign transaction with previous output script.**

In [ ]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 

signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_0 "$previous_output_script" $my_tx)


**Set the input script into the finalised transaction.**

In [ ]:
# bx input-set "[signature] [public key point]" [transaction template]

# my_tx=

my_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_1h_0_0]" $my_tx)


## 3. Validate & Broadcast the endorsed transaction

In [ ]:
# Validate.
bx validate-tx $my_tx

In [ ]:
# Send.
bx send-tx $my_tx

In [ ]:
# Decode transaction for inspection.
bx tx-decode -f json $my_tx | jq